In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from scipy import stats
import datetime

In [2]:
#Importing Files
#37 and 39 refer to the location, 37 is Easter Island, and 39 is a nearby seamount called Pukao.
#The Hab files are the habitat analysis that I did, CTD contains the CTD data by timecode, and DT contains the latitude and longitude data by timecode.
Hab37 = pd.read_csv('37 - Habitat Analysis.csv')
Hab39 = pd.read_csv('39 - Habitat Analysis.csv')
CTD37 = pd.read_csv('37_CTD.csv')
CTD39 = pd.read_csv('39_CTD.csv')
DT37 = pd.read_csv('37_DT.csv')
DT39 = pd.read_csv('39_DT.csv')

2 research sites - 37 (SPG6, a seamount) and 39 (SPG7, Easter Island). The Hab37 and Hab39 dataframes contain the habitat analysis - substrate type, texture, bioturbation, etc. CTD37 and CTD39 are the dataframes with the CTD data, and DT37 and DT39 contain the latitude, longitude data and depth position of the deeptow camera unit at each point.

In [3]:
#Looking at the file type
Hab37?

Type:        DataFrame
String form:
Image       DATE      TIME  \
           0   EPIC_Cruise_MR1 <...>         NaN
           87          NaN         NaN
           88          NaN         NaN
           
           [89 rows x 20 columns]
Length:      89
File:        c:\users\kara\anaconda3\lib\site-packages\pandas\core\frame.py
Docstring:  
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, or list-like objects.

    .. versionchanged:: 0.23.0
       If data is a dict, column order follows insertion-order for
       Python 3.6 and later.

    .. versionchanged:: 0.25.0
       If data is a list of dicts, colu

In [4]:
#Looking at the data
Hab37

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",M - physical processes,M - chemical processes,"M - biological processes (bioturbation, % cover, communities)",M - anthropogenic processes,Altitude,CPC?,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:10:29,Seamount - EI (SPG7),Flat sandy bottom,coarse sand,NaN,regular,NaN,NaN,2D ripples,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
1,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:11:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,4.2,NaN,NaN,NaN,NaN,NaN
2,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:12:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,6.8,NaN,NaN,NaN,NaN,NaN
3,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:13:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - dusting,NaN,NaN,NaN,NaN,NaN,6.8,Y,NaN,NaN,NaN,NaN
4,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:14:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - thin,NaN,NaN,NaN,NaN,NaN,10.3,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:39:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,urchins,NaN,3.0,NaN,NaN,NaN,NaN,NaN
85,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:40:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,"urchins, fish",NaN,3.3,NaN,NaN,NaN,NaN,NaN
86,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:41:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,"urchins, fish",NaN,3.4,NaN,NaN,NaN,NaN,NaN
87,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:42:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,urchins,NaN,2.9,NaN,NaN,NaN,NaN,NaN


In [5]:
CTD37

,DATE,TIME,ALT,CTD_COND,CTD_TEMP,CTD_DEPTH,CTD_SAL,HEAD(GYRO),ROLL(GYRO),PITCH(GYRO)
0,2/13/2019,19:42:49,---.-,5.21224,24.8125,14.16,34.4356,1.7,-1.1,3.1
1,2/13/2019,19:42:50,---.-,5.20695,24.8138,13.39,34.3955,358.8,-0.3,4.1
2,2/13/2019,19:42:51,---.-,5.23634,24.8127,13.25,34.6148,0.1,-1.5,6.7
3,2/13/2019,19:42:52,---.-,5.27145,24.8117,13.58,34.8766,0.1,-1.5,6.7
4,2/13/2019,19:42:53,---.-,5.23748,24.8128,14.14,34.6228,359.4,-1.2,9.5
...,...,...,...,...,...,...,...,...,...,...
7809,2/13/2019,21:52:58,---.-,5.46090,24.8806,14.63,36.2353,17.6,-0.9,9.6
7810,2/13/2019,21:52:59,---.-,5.46052,24.8773,14.97,36.2349,18.1,0.0,5.4
7811,2/13/2019,21:53:00,---.-,5.46184,24.8861,15.28,36.2377,18.4,0.9,0.4
7812,2/13/2019,21:53:01,---.-,5.46234,24.8902,15.12,36.2381,18.4,1.0,1.3


In [6]:
DT39

,DATE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,19:36:42,T1,1,X=8.6,Y=-200.7,27-04.0653 S,109-10.0314 W,Z=22.6,HR=88.0,BRG=178.9,SR=90.8
1,19:39:12,T1,1,X=29.1,Y=-194.1,27-04.0542 S,109-10.0274 W,Z=11.6,HR=90.2,BRG=178.7,SR=91.0
2,19:39:24,T1,1,X=28.3,Y=-194.9,27-04.0547 S,109-10.0279 W,Z=23.9,HR=89.1,BRG=179.0,SR=92.2
3,19:39:30,T1,1,X=27.7,Y=-193.8,27-04.0550 S,109-10.0272 W,Z=18.7,HR=90.2,BRG=179.1,SR=92.1
4,19:39:36,T1,1,X=26.9,Y=-190.8,27-04.0554 S,109-10.0254 W,Z=19.0,HR=93.2,BRG=179.4,SR=95.1
...,...,...,...,...,...,...,...,...,...,...,...
1630,22:45:42,T1,1,X=424.1,Y=-3801.7,27-03.8400 S,109-12.2095 W,Z=46.1,HR=90.9,BRG=180.3,SR=101.9
1631,22:45:48,T1,1,X=426.6,Y=-3805.0,27-03.8387 S,109-12.2114 W,Z=40.5,HR=89.4,BRG=179.0,SR=98.1
1632,22:45:54,T1,1,X=426.9,Y=-3806.9,27-03.8385 S,109-12.2126 W,Z=33.9,HR=91.2,BRG=179.6,SR=97.3
1633,22:46:00,T1,1,X=428.9,Y=-3808.4,27-03.8374 S,109-12.2135 W,Z=29.9,HR=92.3,BRG=178.9,SR=97.0


In [7]:
#Converting all of the files time and date columns to a single datetime object for each dataframe in the following lines.
Hab37['dte'] = Hab37['DATE'] + str(' ') + Hab37['TIME']

In [8]:
Hab37['dte'] = pd.to_datetime(Hab37['dte'], format='%m/%d/%Y %H:%M:%S')

In [9]:
#Me realizing a week later I should have set the datetimes as my indices.
Hab37 = Hab37.set_index(['dte'])

In [10]:
Hab39['dte'] = Hab39['DATE'] + str(' ') + Hab39['TIME']

In [11]:
Hab39['dte'] = pd.to_datetime(Hab39['dte'], format='%m/%d/%Y %H:%M:%S')

In [12]:
Hab39 = Hab39.set_index(['dte'])

In [13]:
CTD37['dte'] = CTD37['DATE'] + str(' ') + CTD37['TIME']

In [14]:
CTD37['dte'] = pd.to_datetime(CTD37['dte'], format='%m/%d/%Y %H:%M:%S')

In [15]:
CTD37 = CTD37.set_index(['dte'])

In [16]:
CTD39['dte'] = CTD39['DATE'] + str(' ') + CTD39['TIME']

In [17]:
CTD39['dte'] = pd.to_datetime(CTD39['dte'], format='%m/%d/%Y %H:%M:%S')

In [18]:
CTD39 = CTD39.set_index(['dte'])

In [19]:
DT37['dte'] = str('2/13/2019') + str(' ') + DT37['DATE']

In [20]:
DT37['dte'] = pd.to_datetime(DT37['dte'], format='%m/%d/%Y %H:%M:%S')

In [21]:
DT37 = DT37.set_index(['dte'])

In [22]:
DT39['dte'] = str('2/16/2019') + str(' ') + DT39['DATE']

In [23]:
DT39['dte'] = pd.to_datetime(DT39['dte'], format='%m/%d/%Y %H:%M:%S')

In [24]:
DT39 = DT39.set_index(['dte'])

In [25]:
#The latitude and longitude files are in degrees and minutes, so I converted them to degrees only. To do this I first split the data into two columns using the hyphen separating the degrees and minutes. This created a new dataframe 'split,' and I did this for both latitude and longitude.
split = DT37['Latitude'].str.split('-', n = 1, expand = True)
split_lon = DT37['Longitude'].str.split('-', n = 1, expand = True)

In [26]:
#Then used str.rstrip() to remove the remaining non-numerical characters from the string.
split[1] = split[1].str.rstrip(' ,S')
split_lon[1] = split_lon[1].str.rstrip(' ,W')

In [27]:
#And then converted the columns to integers and floats as needed for the calculations.
split[1] = split[1].astype(float)
split[1]

dte
2019-02-13 19:41:54     9.8848
2019-02-13 19:42:30     9.8871
2019-02-13 19:43:18    11.3013
2019-02-13 19:43:30    11.2994
2019-02-13 19:43:36    11.2989
                        ...   
2019-02-13 21:52:06    10.8641
2019-02-13 21:52:12    10.8634
2019-02-13 21:52:18    10.8642
2019-02-13 21:52:24    10.8628
2019-02-13 21:52:30    10.8616
Name: 1, Length: 1113, dtype: float64

In [28]:
split[0] = split[0].astype(int)
split[0]

dte
2019-02-13 19:41:54    26
2019-02-13 19:42:30    26
2019-02-13 19:43:18    26
2019-02-13 19:43:30    26
2019-02-13 19:43:36    26
                       ..
2019-02-13 21:52:06    26
2019-02-13 21:52:12    26
2019-02-13 21:52:18    26
2019-02-13 21:52:24    26
2019-02-13 21:52:30    26
Name: 0, Length: 1113, dtype: int32

In [29]:
#Calculating the latitude and adding it back to the original dataframe as a new column.
DT37['LAT'] = -1*((split[0])+((split[1])/60))

In [30]:
#Looking at the new column.
DT37['LAT']

dte
2019-02-13 19:41:54   -26.164747
2019-02-13 19:42:30   -26.164785
2019-02-13 19:43:18   -26.188355
2019-02-13 19:43:30   -26.188323
2019-02-13 19:43:36   -26.188315
                         ...    
2019-02-13 21:52:06   -26.181068
2019-02-13 21:52:12   -26.181057
2019-02-13 21:52:18   -26.181070
2019-02-13 21:52:24   -26.181047
2019-02-13 21:52:30   -26.181027
Name: LAT, Length: 1113, dtype: float64

In [31]:
#And then dropping the NaN values.
DT37['LAT'].dropna()

dte
2019-02-13 19:41:54   -26.164747
2019-02-13 19:42:30   -26.164785
2019-02-13 19:43:18   -26.188355
2019-02-13 19:43:30   -26.188323
2019-02-13 19:43:36   -26.188315
                         ...    
2019-02-13 21:52:06   -26.181068
2019-02-13 21:52:12   -26.181057
2019-02-13 21:52:18   -26.181070
2019-02-13 21:52:24   -26.181047
2019-02-13 21:52:30   -26.181027
Name: LAT, Length: 1113, dtype: float64

In [32]:
#And nowdoing the same for longitude as well.
split_lon[1] = split_lon[1].astype(float)
split_lon[1]

dte
2019-02-13 19:41:54    58.7199
2019-02-13 19:42:30    58.5475
2019-02-13 19:43:18    59.8867
2019-02-13 19:43:30    59.8852
2019-02-13 19:43:36    59.8843
                        ...   
2019-02-13 21:52:06    58.5303
2019-02-13 21:52:12    58.5293
2019-02-13 21:52:18    58.5279
2019-02-13 21:52:24    58.5288
2019-02-13 21:52:30    58.5280
Name: 1, Length: 1113, dtype: float64

In [33]:
split_lon[0] = split_lon[0].astype(int)
split_lon[0]

dte
2019-02-13 19:41:54    102
2019-02-13 19:42:30    102
2019-02-13 19:43:18    102
2019-02-13 19:43:30    102
2019-02-13 19:43:36    102
                      ... 
2019-02-13 21:52:06    102
2019-02-13 21:52:12    102
2019-02-13 21:52:18    102
2019-02-13 21:52:24    102
2019-02-13 21:52:30    102
Name: 0, Length: 1113, dtype: int32

In [34]:
#Calculating the longitude and adding it back to the original dataframe as a new column.
DT37['LON'] = -1*((split_lon[0])+((split_lon[1])/60))

In [35]:
DT37['LON']

dte
2019-02-13 19:41:54   -102.978665
2019-02-13 19:42:30   -102.975792
2019-02-13 19:43:18   -102.998112
2019-02-13 19:43:30   -102.998087
2019-02-13 19:43:36   -102.998072
                          ...    
2019-02-13 21:52:06   -102.975505
2019-02-13 21:52:12   -102.975488
2019-02-13 21:52:18   -102.975465
2019-02-13 21:52:24   -102.975480
2019-02-13 21:52:30   -102.975467
Name: LON, Length: 1113, dtype: float64

In [36]:
#Following the same steps again for DT39.
split = DT39['Latitude'].str.split('-', n = 1, expand = True)
split_lon = DT39['Longitude'].str.split('-', n = 1, expand = True)

In [37]:
split[1] = split[1].str.rstrip(' ,S')
split_lon[1] = split_lon[1].str.rstrip(' ,W')

In [38]:
split[1] = split[1].astype(float)
split[1]

dte
2019-02-16 19:36:42    4.0653
2019-02-16 19:39:12    4.0542
2019-02-16 19:39:24    4.0547
2019-02-16 19:39:30    4.0550
2019-02-16 19:39:36    4.0554
                        ...  
2019-02-16 22:45:42    3.8400
2019-02-16 22:45:48    3.8387
2019-02-16 22:45:54    3.8385
2019-02-16 22:46:00    3.8374
2019-02-16 22:46:06    3.8364
Name: 1, Length: 1635, dtype: float64

In [39]:
split[0] = split[0].astype(float)
split[0]

dte
2019-02-16 19:36:42    27.0
2019-02-16 19:39:12    27.0
2019-02-16 19:39:24    27.0
2019-02-16 19:39:30    27.0
2019-02-16 19:39:36    27.0
                       ... 
2019-02-16 22:45:42    27.0
2019-02-16 22:45:48    27.0
2019-02-16 22:45:54    27.0
2019-02-16 22:46:00    27.0
2019-02-16 22:46:06    27.0
Name: 0, Length: 1635, dtype: float64

In [40]:
DT39['LAT'] = -1*((split[0])+((split[1])/60))

In [41]:
DT39['LAT'] = DT39['LAT'].dropna()

In [42]:
DT39['LAT']

dte
2019-02-16 19:36:42   -27.067755
2019-02-16 19:39:12   -27.067570
2019-02-16 19:39:24   -27.067578
2019-02-16 19:39:30   -27.067583
2019-02-16 19:39:36   -27.067590
                         ...    
2019-02-16 22:45:42   -27.064000
2019-02-16 22:45:48   -27.063978
2019-02-16 22:45:54   -27.063975
2019-02-16 22:46:00   -27.063957
2019-02-16 22:46:06   -27.063940
Name: LAT, Length: 1635, dtype: float64

In [43]:
split_lon[1] = split_lon[1].astype(float)
split_lon[1]

dte
2019-02-16 19:36:42    10.0314
2019-02-16 19:39:12    10.0274
2019-02-16 19:39:24    10.0279
2019-02-16 19:39:30    10.0272
2019-02-16 19:39:36    10.0254
                        ...   
2019-02-16 22:45:42    12.2095
2019-02-16 22:45:48    12.2114
2019-02-16 22:45:54    12.2126
2019-02-16 22:46:00    12.2135
2019-02-16 22:46:06    12.2158
Name: 1, Length: 1635, dtype: float64

In [44]:
split_lon[0] = split_lon[0].astype(float)
split_lon[0]

dte
2019-02-16 19:36:42    109.0
2019-02-16 19:39:12    109.0
2019-02-16 19:39:24    109.0
2019-02-16 19:39:30    109.0
2019-02-16 19:39:36    109.0
                       ...  
2019-02-16 22:45:42    109.0
2019-02-16 22:45:48    109.0
2019-02-16 22:45:54    109.0
2019-02-16 22:46:00    109.0
2019-02-16 22:46:06    109.0
Name: 0, Length: 1635, dtype: float64

In [45]:
#Calculating the longitude and adding it back to the original dataframe as a new column.
DT39['LON'] = -1*((split_lon[0])+((split_lon[1])/60))

In [46]:
DT39['LON']

dte
2019-02-16 19:36:42   -109.167190
2019-02-16 19:39:12   -109.167123
2019-02-16 19:39:24   -109.167132
2019-02-16 19:39:30   -109.167120
2019-02-16 19:39:36   -109.167090
                          ...    
2019-02-16 22:45:42   -109.203492
2019-02-16 22:45:48   -109.203523
2019-02-16 22:45:54   -109.203543
2019-02-16 22:46:00   -109.203558
2019-02-16 22:46:06   -109.203597
Name: LON, Length: 1635, dtype: float64

In [47]:
#A look at the entire dataframe, now with the LAT and LON columns which can be mapped!
DT37

,DATE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10,LAT,LON
dte,,,,,,,,,,,,,
2019-02-13 19:41:54,19:41:54,T1,1,X=2059.2,Y=1299.8,26-09.8848 S,102-58.7199 W,Z=1467.1,HR=3189.6,BRG=341.5,SR=3510.9,-26.164747,-102.978665
2019-02-13 19:42:30,19:42:30,T1,1,X=2054.9,Y=1587.1,26-09.8871 S,102-58.5475 W,Z=1077.5,HR=3355.3,BRG=345.2,SR=3524.1,-26.164785,-102.975792
2019-02-13 19:43:18,19:43:18,T1,1,X=-556.4,Y=-644.3,26-11.3013 S,102-59.8867 W,Z=23.4,HR=93.4,BRG=178.1,SR=96.3,-26.188355,-102.998112
2019-02-13 19:43:30,19:43:30,T1,1,X=-552.9,Y=-641.7,26-11.2994 S,102-59.8852 W,Z=23.6,HR=93.3,BRG=178.4,SR=96.2,-26.188323,-102.998087
2019-02-13 19:43:36,19:43:36,T1,1,X=-552.0,Y=-640.2,26-11.2989 S,102-59.8843 W,Z=26.0,HR=93.4,BRG=178.4,SR=96.9,-26.188315,-102.998072
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-13 21:52:06,21:52:06,T1,1,X=250.8,Y=1615.6,26-10.8641 S,102-58.5303 W,Z=27.8,HR=91.4,BRG=179.2,SR=95.5,-26.181068,-102.975505
2019-02-13 21:52:12,21:52:12,T1,1,X=252.2,Y=1617.2,26-10.8634 S,102-58.5293 W,Z=29.8,HR=91.3,BRG=178.8,SR=96.1,-26.181057,-102.975488
2019-02-13 21:52:18,21:52:18,T1,1,X=250.7,Y=1619.5,26-10.8642 S,102-58.5279 W,Z=27.2,HR=91.5,BRG=177.5,SR=95.5,-26.181070,-102.975465


In [48]:
DT39

,DATE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10,LAT,LON
dte,,,,,,,,,,,,,
2019-02-16 19:36:42,19:36:42,T1,1,X=8.6,Y=-200.7,27-04.0653 S,109-10.0314 W,Z=22.6,HR=88.0,BRG=178.9,SR=90.8,-27.067755,-109.167190
2019-02-16 19:39:12,19:39:12,T1,1,X=29.1,Y=-194.1,27-04.0542 S,109-10.0274 W,Z=11.6,HR=90.2,BRG=178.7,SR=91.0,-27.067570,-109.167123
2019-02-16 19:39:24,19:39:24,T1,1,X=28.3,Y=-194.9,27-04.0547 S,109-10.0279 W,Z=23.9,HR=89.1,BRG=179.0,SR=92.2,-27.067578,-109.167132
2019-02-16 19:39:30,19:39:30,T1,1,X=27.7,Y=-193.8,27-04.0550 S,109-10.0272 W,Z=18.7,HR=90.2,BRG=179.1,SR=92.1,-27.067583,-109.167120
2019-02-16 19:39:36,19:39:36,T1,1,X=26.9,Y=-190.8,27-04.0554 S,109-10.0254 W,Z=19.0,HR=93.2,BRG=179.4,SR=95.1,-27.067590,-109.167090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-16 22:45:42,22:45:42,T1,1,X=424.1,Y=-3801.7,27-03.8400 S,109-12.2095 W,Z=46.1,HR=90.9,BRG=180.3,SR=101.9,-27.064000,-109.203492
2019-02-16 22:45:48,22:45:48,T1,1,X=426.6,Y=-3805.0,27-03.8387 S,109-12.2114 W,Z=40.5,HR=89.4,BRG=179.0,SR=98.1,-27.063978,-109.203523
2019-02-16 22:45:54,22:45:54,T1,1,X=426.9,Y=-3806.9,27-03.8385 S,109-12.2126 W,Z=33.9,HR=91.2,BRG=179.6,SR=97.3,-27.063975,-109.203543


In [49]:
DT39 = DT39.dropna()
DT39

,DATE,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Latitude,Longitude,Depth,Unnamed: 8,Unnamed: 9,Unnamed: 10,LAT,LON
dte,,,,,,,,,,,,,
2019-02-16 19:36:42,19:36:42,T1,1,X=8.6,Y=-200.7,27-04.0653 S,109-10.0314 W,Z=22.6,HR=88.0,BRG=178.9,SR=90.8,-27.067755,-109.167190
2019-02-16 19:39:12,19:39:12,T1,1,X=29.1,Y=-194.1,27-04.0542 S,109-10.0274 W,Z=11.6,HR=90.2,BRG=178.7,SR=91.0,-27.067570,-109.167123
2019-02-16 19:39:24,19:39:24,T1,1,X=28.3,Y=-194.9,27-04.0547 S,109-10.0279 W,Z=23.9,HR=89.1,BRG=179.0,SR=92.2,-27.067578,-109.167132
2019-02-16 19:39:30,19:39:30,T1,1,X=27.7,Y=-193.8,27-04.0550 S,109-10.0272 W,Z=18.7,HR=90.2,BRG=179.1,SR=92.1,-27.067583,-109.167120
2019-02-16 19:39:36,19:39:36,T1,1,X=26.9,Y=-190.8,27-04.0554 S,109-10.0254 W,Z=19.0,HR=93.2,BRG=179.4,SR=95.1,-27.067590,-109.167090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-16 22:45:42,22:45:42,T1,1,X=424.1,Y=-3801.7,27-03.8400 S,109-12.2095 W,Z=46.1,HR=90.9,BRG=180.3,SR=101.9,-27.064000,-109.203492
2019-02-16 22:45:48,22:45:48,T1,1,X=426.6,Y=-3805.0,27-03.8387 S,109-12.2114 W,Z=40.5,HR=89.4,BRG=179.0,SR=98.1,-27.063978,-109.203523
2019-02-16 22:45:54,22:45:54,T1,1,X=426.9,Y=-3806.9,27-03.8385 S,109-12.2126 W,Z=33.9,HR=91.2,BRG=179.6,SR=97.3,-27.063975,-109.203543


In [50]:
#Example code from online on how to find the nearest datetime object
#dt = pd.to_datetime(“2016–04–23 11:00:00”)
#df.index.get_loc(dt, method=“nearest”)
##get index date
#idx = df.index[df.index.get_loc(dt, method='nearest')]
##row to series
#s = df.iloc[df.index.get_loc(dt, method='nearest')]

##Failed attempt 4-thousand
#DT37_0 = DT37['dte'][0]()[DT37['dte'][0]().get_loc(DT37['dte'][0], method='nearest')]

#idx = df.index[df.index.get_loc(dt, method='nearest')]
DT37_0 = DT37.index[DT37.index.get_loc(Hab37.index[0], method='nearest')]
DT37_1 = DT37.iloc[DT37.index.get_loc(Hab37.index[0], method='nearest')]

In [51]:
DT37_0

#It worked! So this is the closes timestamp from DT37 (which has lat long values associated with it) to the first index from Hab37, which is associated with one of my photos!
#Next step - make a function to do this for each of the images to pull out the rows of lat long data from DT and CTD rows from CTD files for each index in the Hab files with the photos. Then can map the survey transects.

Timestamp('2019-02-13 20:10:42')

In [52]:
DT37_1

DATE                20:10:42
Unnamed: 1                T1
Unnamed: 2                 1
Unnamed: 3          X=-460.3
Unnamed: 4          Y=-400.1
Latitude        26-11.2493 S
Longitude      102-59.7401 W
Depth                Z=883.1
Unnamed: 8          HR=130.2
Unnamed: 9         BRG=183.0
Unnamed: 10         SR=892.7
LAT                 -26.1875
LON                 -102.996
Name: 2019-02-13 20:10:42, dtype: object

In [53]:
Data37 = Data37.T

NameError: name 'Data37' is not defined

In [ ]:
#And... scrapping this method

#Function - lets pseudocode it

#use my index code
#add the index by one each time (89 rows)
#for each for timestamp add the row to a new data frame


    
num = 1
Data37 = []
    
while num < 89:
    
    new_series = DT37.iloc[DT37.index.get_loc(Hab37.index[num], method='nearest')]
    
    Data37.append(pd.DataFrame(new_series))
    
    nextnum = num + 1
        
    next_series = DT37.iloc[DT37.index.get_loc(Hab37.index[nextnum], method='nearest')]
    
    Data37.append(pd.DataFrame(next_series))
    
    num = nextnum +1
    
print(Data37)
    

In [ ]:
DT37.iloc[DT37.index.get_loc(Hab37.index[0], method='nearest')][[0, 11, 12]]


In [54]:
DT37.iloc[DT37.index.get_loc(Hab37.index[1], method='nearest')][[0, 11, 12]]


DATE    20:11:18
LAT     -26.1875
LON     -102.996
Name: 2019-02-13 20:11:18, dtype: object

In [55]:
DT37.iloc[DT37.index.get_loc(Hab37.index[2], method='nearest')][[0, 11, 12]]


DATE    20:12:18
LAT     -26.1874
LON     -102.995
Name: 2019-02-13 20:12:18, dtype: object

In [56]:
Hab_2 = []
num = 0
Hab37_2 = DT37.iloc[DT37.index.get_loc(Hab37.index[num], method='nearest')][[0, 11, 12]]
Hab37_2

DATE    20:10:42
LAT     -26.1875
LON     -102.996
Name: 2019-02-13 20:10:42, dtype: object

In [59]:
#New method - starting with just getting the index time from the DT file

Hab_37 = []
num = 0
while num < 89:

    Hab37_2 = DT37.index[DT37.index.get_loc(Hab37.index[num], method='nearest')]
    Hab_37.append(Hab37_2)
    num = num +1
print(Hab_37)

[Timestamp('2019-02-13 20:10:42'), Timestamp('2019-02-13 20:11:18'), Timestamp('2019-02-13 20:12:18'), Timestamp('2019-02-13 20:13:18'), Timestamp('2019-02-13 20:14:18'), Timestamp('2019-02-13 20:15:18'), Timestamp('2019-02-13 20:16:18'), Timestamp('2019-02-13 20:17:18'), Timestamp('2019-02-13 20:18:18'), Timestamp('2019-02-13 20:19:18'), Timestamp('2019-02-13 20:20:18'), Timestamp('2019-02-13 20:21:18'), Timestamp('2019-02-13 20:22:12'), Timestamp('2019-02-13 20:23:18'), Timestamp('2019-02-13 20:24:18'), Timestamp('2019-02-13 20:25:24'), Timestamp('2019-02-13 20:26:18'), Timestamp('2019-02-13 20:27:18'), Timestamp('2019-02-13 20:28:18'), Timestamp('2019-02-13 20:29:18'), Timestamp('2019-02-13 20:30:18'), Timestamp('2019-02-13 20:31:12'), Timestamp('2019-02-13 20:32:18'), Timestamp('2019-02-13 20:33:18'), Timestamp('2019-02-13 20:34:12'), Timestamp('2019-02-13 20:35:18'), Timestamp('2019-02-13 20:36:18'), Timestamp('2019-02-13 20:37:18'), Timestamp('2019-02-13 20:38:18'), Timestamp('20

In [60]:
#Adding that time to the Hab37 dataframe

Hab37['LTIME'] = Hab_37

In [61]:
Hab37

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",...,M - chemical processes,"M - biological processes (bioturbation, % cover, communities)",M - anthropogenic processes,Altitude,CPC?,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,LTIME
dte,,,,,,,,,,,,,,,,,,,,,
2019-02-13 20:10:29,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:10:29,Seamount - EI (SPG7),Flat sandy bottom,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:10:42
2019-02-13 20:11:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:11:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,4.2,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:11:18
2019-02-13 20:12:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:12:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,6.8,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:12:18
2019-02-13 20:13:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:13:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - dusting,NaN,...,NaN,NaN,NaN,6.8,Y,NaN,NaN,NaN,NaN,2019-02-13 20:13:18
2019-02-13 20:14:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:14:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - thin,NaN,...,NaN,NaN,NaN,10.3,Y,NaN,NaN,NaN,NaN,2019-02-13 20:14:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-13 21:39:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:39:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,urchins,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:39:18
2019-02-13 21:40:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:40:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,"urchins, fish",NaN,3.3,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:40:18
2019-02-13 21:41:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:41:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,"urchins, fish",NaN,3.4,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:41:18


In [62]:
#Now getting the series with the latitude and longitude data, but they'll need some cleaning up.

Hab_37 = []
num = 0
while num < 89:

    Hab37_2 = DT37.iloc[DT37.index.get_loc(Hab37.index[num], method='nearest')][[11, 12]]
    Hab_37.append(Hab37_2)
    num = num +1
print(Hab_37)

[LAT   -26.1875
LON   -102.996
Name: 2019-02-13 20:10:42, dtype: object, LAT   -26.1875
LON   -102.996
Name: 2019-02-13 20:11:18, dtype: object, LAT   -26.1874
LON   -102.995
Name: 2019-02-13 20:12:18, dtype: object, LAT   -26.1873
LON   -102.995
Name: 2019-02-13 20:13:18, dtype: object, LAT   -26.1872
LON   -102.995
Name: 2019-02-13 20:14:18, dtype: object, LAT   -26.1871
LON   -102.995
Name: 2019-02-13 20:15:18, dtype: object, LAT   -26.1871
LON   -102.995
Name: 2019-02-13 20:16:18, dtype: object, LAT    -26.187
LON   -102.994
Name: 2019-02-13 20:17:18, dtype: object, LAT   -26.1868
LON   -102.994
Name: 2019-02-13 20:18:18, dtype: object, LAT   -26.1868
LON   -102.994
Name: 2019-02-13 20:19:18, dtype: object, LAT   -26.1867
LON   -102.994
Name: 2019-02-13 20:20:18, dtype: object, LAT   -26.1866
LON   -102.994
Name: 2019-02-13 20:21:18, dtype: object, LAT   -26.1865
LON   -102.993
Name: 2019-02-13 20:22:12, dtype: object, LAT   -26.1864
LON   -102.993
Name: 2019-02-13 20:23:18, dtype:

In [63]:
#This is my test run, I found out I have to do these one at a time so I'll do a function in a minute.
Hab_37[0]

LAT   -26.1875
LON   -102.996
Name: 2019-02-13 20:10:42, dtype: object

In [64]:
#First making a column for the series in Hab37

Hab37['LDATE'] = Hab_37

In [65]:
#Making the objects strings to work with more easily
st = str(Hab37['LDATE'][0])

In [66]:
#splitting them into rows, lat and lon
spl = st.split('\n')

In [67]:
#deleting the extra unnecessary row

del(spl[2])

In [68]:
#making it a dataframe so it can be added to the Hab37 dataframe in a minute
spl = pd.DataFrame(spl)

In [79]:
#Removing the letters and spaces so it'll be a float rather than string
spl = spl.str.strip(' ,L,A,T,O,N')
spl

0    -26.1813
1    -102.976
Name: 0, dtype: object

In [80]:
#Creating my lat and lon columns in the Hab37 dataframe
Hab37['LAT'] = ''
Hab37['LON'] = ''

In [81]:
#and then adding the respective data
Hab37['LAT'][0] = spl[0]
Hab37['LON'][0] = spl[1]

<ipython-input-81-3447f5b73db0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['LAT'][0] = spl[0]
<ipython-input-81-3447f5b73db0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['LON'][0] = spl[1]


In [82]:
Hab37

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",...,Altitude,CPC?,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,LTIME,LDATE,LAT,LON
dte,,,,,,,,,,,,,,,,,,,,,
2019-02-13 20:10:29,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:10:29,Seamount - EI (SPG7),Flat sandy bottom,coarse sand,NaN,regular,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:10:42,LAT -26.1875 LON -102.996 Name: 2019-02-13...,-26.1813,-102.976
2019-02-13 20:11:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:11:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,4.2,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:11:18,LAT -26.1875 LON -102.996 Name: 2019-02-13...,,
2019-02-13 20:12:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:12:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,6.8,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:12:18,LAT -26.1874 LON -102.995 Name: 2019-02-13...,,
2019-02-13 20:13:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:13:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - dusting,NaN,...,6.8,Y,NaN,NaN,NaN,NaN,2019-02-13 20:13:18,LAT -26.1873 LON -102.995 Name: 2019-02-13...,,
2019-02-13 20:14:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:14:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - thin,NaN,...,10.3,Y,NaN,NaN,NaN,NaN,2019-02-13 20:14:18,LAT -26.1872 LON -102.995 Name: 2019-02-13...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-13 21:39:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:39:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:39:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,,
2019-02-13 21:40:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:40:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,3.3,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:40:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,,
2019-02-13 21:41:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:41:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,3.4,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:41:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,,


In [89]:
##IT WORKS!!! Only took 2 weeks...... Now to do the same for Hab39 and then both CTD files.
num = 0
Hab37['LAT'] = ''
Hab37['LON'] = ''
    
while num <89:

    Hab37['LDATE'] = Hab_37
    st = str(Hab37['LDATE'][num])
    spl = st.split('\n')
    del(spl[2])
    spl = pd.DataFrame(spl)
    spl = spl[0].str.strip(' ,L,A,T,O,N')
    Hab37['LAT'][num] = spl[0]
    Hab37['LON'][num] = spl[1]
   
    num = num + 1
print(Hab37)

<ipython-input-89-b51ca0cdcec2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['LAT'][num] = spl[0]
<ipython-input-89-b51ca0cdcec2>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['LON'][num] = spl[1]


                                                                 Image  \
dte                                                                      
2019-02-13 20:10:29  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:11:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:12:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:13:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:14:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
...                                                                ...   
2019-02-13 21:39:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:40:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:41:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:42:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:43:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   

                          DATE      T

In [90]:
Hab37['LAT']

dte
2019-02-13 20:10:29    -26.1875
2019-02-13 20:11:17    -26.1875
2019-02-13 20:12:17    -26.1874
2019-02-13 20:13:17    -26.1873
2019-02-13 20:14:17    -26.1872
                         ...   
2019-02-13 21:39:18    -26.1813
2019-02-13 21:40:18    -26.1813
2019-02-13 21:41:18    -26.1813
2019-02-13 21:42:18    -26.1813
2019-02-13 21:43:18    -26.1813
Name: LAT, Length: 89, dtype: object

In [92]:
#Same thing, Hab39

Hab_39 = []
num = 0
while num < 150:

    Hab39_2 = DT39.index[DT39.index.get_loc(Hab39.index[num], method='nearest')]
    Hab_39.append(Hab39_2)
    num = num +1
print(Hab_39)

[Timestamp('2019-02-16 20:04:00'), Timestamp('2019-02-16 20:05:00'), Timestamp('2019-02-16 20:06:00'), Timestamp('2019-02-16 20:07:00'), Timestamp('2019-02-16 20:08:00'), Timestamp('2019-02-16 20:09:00'), Timestamp('2019-02-16 20:10:00'), Timestamp('2019-02-16 20:11:00'), Timestamp('2019-02-16 20:11:54'), Timestamp('2019-02-16 20:13:00'), Timestamp('2019-02-16 20:14:00'), Timestamp('2019-02-16 20:15:00'), Timestamp('2019-02-16 20:16:00'), Timestamp('2019-02-16 20:17:00'), Timestamp('2019-02-16 20:18:00'), Timestamp('2019-02-16 20:19:00'), Timestamp('2019-02-16 20:20:00'), Timestamp('2019-02-16 20:21:00'), Timestamp('2019-02-16 20:22:00'), Timestamp('2019-02-16 20:23:00'), Timestamp('2019-02-16 20:24:00'), Timestamp('2019-02-16 20:25:00'), Timestamp('2019-02-16 20:26:06'), Timestamp('2019-02-16 20:27:00'), Timestamp('2019-02-16 20:28:00'), Timestamp('2019-02-16 20:29:00'), Timestamp('2019-02-16 20:30:00'), Timestamp('2019-02-16 20:31:06'), Timestamp('2019-02-16 20:32:06'), Timestamp('20

In [93]:
#Adding that time to the Hab37 dataframe

Hab39['LTIME'] = Hab_39
Hab39

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",M - physical processes,M - chemical processes,"M - biological processes (bioturbation, % cover, communities)",M - anthropogenic processes,Altitude,CPC?,Unnamed: 16,LTIME
dte,,,,,,,,,,,,,,,,,,
2019-02-16 20:03:58,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,20:03:58,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,2019-02-16 20:04:00
2019-02-16 20:04:58,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,20:04:58,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,4.4,NaN,NaN,2019-02-16 20:05:00
2019-02-16 20:05:58,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,20:05:58,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,4.9,NaN,NaN,2019-02-16 20:06:00
2019-02-16 20:06:58,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,20:06:58,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,5.3,NaN,NaN,2019-02-16 20:07:00
2019-02-16 20:07:58,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,20:07:58,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,regular,NaN,NaN,NaN,NaN,NaN,NaN,5.6,NaN,NaN,2019-02-16 20:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-16 22:31:02,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,22:31:02,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,irregular,erosional features,NaN,NaN,NaN,coral?,NaN,4.5,?,NaN,2019-02-16 22:31:00
2019-02-16 22:32:02,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,22:32:02,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,irregular,erosional features,NaN,NaN,NaN,"coral?, rubble",NaN,3.2,?,NaN,2019-02-16 22:32:00
2019-02-16 22:33:02,EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...,2/16/2019,22:33:02,Seamount SPG7a (Pukao),NaN,coarse sand,NaN,irregular,erosional features,NaN,NaN,NaN,"coral?, crab, rubble",NaN,4.4,?,NaN,2019-02-16 22:33:00


In [97]:
Hab_39 = []
num = 0
while num < 150:

    Hab39_2 = DT39.iloc[DT39.index.get_loc(Hab39.index[num], method='nearest')][[0, 11, 12]]
    Hab_39.append(Hab39_2)
    num = num +1
print(Hab_39)

[DATE    20:04:00
LAT     -27.0671
LON     -109.169
Name: 2019-02-16 20:04:00, dtype: object, DATE    20:05:00
LAT     -27.0671
LON     -109.169
Name: 2019-02-16 20:05:00, dtype: object, DATE    20:06:00
LAT     -27.0675
LON     -109.169
Name: 2019-02-16 20:06:00, dtype: object, DATE    20:07:00
LAT     -27.0671
LON     -109.169
Name: 2019-02-16 20:07:00, dtype: object, DATE    20:08:00
LAT      -27.067
LON     -109.169
Name: 2019-02-16 20:08:00, dtype: object, DATE    20:09:00
LAT      -27.067
LON     -109.169
Name: 2019-02-16 20:09:00, dtype: object, DATE    20:10:00
LAT      -27.067
LON     -109.169
Name: 2019-02-16 20:10:00, dtype: object, DATE    20:11:00
LAT     -27.0669
LON      -109.17
Name: 2019-02-16 20:11:00, dtype: object, DATE    20:11:54
LAT     -27.0669
LON      -109.17
Name: 2019-02-16 20:11:54, dtype: object, DATE    20:13:00
LAT     -27.0668
LON      -109.17
Name: 2019-02-16 20:13:00, dtype: object, DATE    20:14:00
LAT     -27.0668
LON      -109.17
Name: 2019-02-16 2

In [109]:
#Only required a small modification, but now it's good. These loops are definitely getting easier, that's for sure.

num = 0
Hab39['LAT'] = ''
Hab39['LON'] = ''

    
while num <150:

    Hab39['LDATE'] = Hab_39
    st = str(Hab39['LDATE'][num])
    spl = st.split('\n')
    del(spl[-1], spl[0])
    spl = pd.DataFrame(spl)
    spl = spl[0].str.strip(' , L, A, T, O, N')
    Hab39['LAT'][num] = spl[0]
    Hab39['LON'][num] = spl[1]
   
    num = num + 1
print(Hab39)

<ipython-input-109-a86102ee24ac>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab39['LAT'][num] = spl[0]
<ipython-input-109-a86102ee24ac>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab39['LON'][num] = spl[1]


                                                                 Image  \
dte                                                                      
2019-02-16 20:03:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:04:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:05:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:06:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:07:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
...                                                                ...   
2019-02-16 22:31:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:32:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:33:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:34:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:35:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   

                          DATE      T

,DATE,TIME,ALT,CTD_COND,CTD_TEMP,CTD_DEPTH,CTD_SAL,HEAD(GYRO),ROLL(GYRO),PITCH(GYRO)
dte,,,,,,,,,,
2019-02-13 19:42:49,2/13/2019,19:42:49,---.-,5.21224,24.8125,14.16,34.4356,1.7,-1.1,3.1
2019-02-13 19:42:50,2/13/2019,19:42:50,---.-,5.20695,24.8138,13.39,34.3955,358.8,-0.3,4.1
2019-02-13 19:42:51,2/13/2019,19:42:51,---.-,5.23634,24.8127,13.25,34.6148,0.1,-1.5,6.7
2019-02-13 19:42:52,2/13/2019,19:42:52,---.-,5.27145,24.8117,13.58,34.8766,0.1,-1.5,6.7
2019-02-13 19:42:53,2/13/2019,19:42:53,---.-,5.23748,24.8128,14.14,34.6228,359.4,-1.2,9.5
...,...,...,...,...,...,...,...,...,...,...
2019-02-13 21:52:58,2/13/2019,21:52:58,---.-,5.46090,24.8806,14.63,36.2353,17.6,-0.9,9.6
2019-02-13 21:52:59,2/13/2019,21:52:59,---.-,5.46052,24.8773,14.97,36.2349,18.1,0.0,5.4
2019-02-13 21:53:00,2/13/2019,21:53:00,---.-,5.46184,24.8861,15.28,36.2377,18.4,0.9,0.4


In [117]:
#Had to add this bit for an issue with non-unique values
CTD37 = CTD37.loc[~CTD37.index.duplicated(keep = 'first')]

In [120]:
#Now to start on the CTD data

Hab_37 = []
num = 0
while num < 89:

    Hab37_2 = CTD37.index[CTD37.index.get_loc(Hab37.index[num], method='nearest')]
    Hab_37.append(Hab37_2)
    num = num +1
print(Hab_37)

[Timestamp('2019-02-13 20:10:29'), Timestamp('2019-02-13 20:11:17'), Timestamp('2019-02-13 20:12:17'), Timestamp('2019-02-13 20:13:17'), Timestamp('2019-02-13 20:14:17'), Timestamp('2019-02-13 20:15:17'), Timestamp('2019-02-13 20:16:17'), Timestamp('2019-02-13 20:17:17'), Timestamp('2019-02-13 20:18:17'), Timestamp('2019-02-13 20:19:17'), Timestamp('2019-02-13 20:20:17'), Timestamp('2019-02-13 20:21:17'), Timestamp('2019-02-13 20:22:17'), Timestamp('2019-02-13 20:23:17'), Timestamp('2019-02-13 20:24:17'), Timestamp('2019-02-13 20:25:17'), Timestamp('2019-02-13 20:26:17'), Timestamp('2019-02-13 20:27:17'), Timestamp('2019-02-13 20:28:17'), Timestamp('2019-02-13 20:29:17'), Timestamp('2019-02-13 20:30:17'), Timestamp('2019-02-13 20:31:17'), Timestamp('2019-02-13 20:32:18'), Timestamp('2019-02-13 20:33:18'), Timestamp('2019-02-13 20:34:18'), Timestamp('2019-02-13 20:35:18'), Timestamp('2019-02-13 20:36:18'), Timestamp('2019-02-13 20:37:18'), Timestamp('2019-02-13 20:38:18'), Timestamp('20

In [128]:
Hab37['CDATE'] = Hab_37
Hab37

,Image,DATE,TIME,"Subsystem (Seamount vs island, seamount top, flank, base, island slope upper <200, intermediate500-1000, lower 1000+)",Class (meso-macrohab - based on seafloor morphology),"Substrate subclass (organic debris, mud, sand, gravel, pebble, cobble, boulder, mixed sediment, bedrock, sedimentary, Tapia: rock, silty sediments, mixed substrate, coarse sand)","Slope subclass(flat 0-5, sloping 5-30, steeply sloping 30-60, vertical 60-90, overhang >90)","M - morphology (regular: homogeneous, little relief; irregular: non-uniform, relief 1-10 m; hummocky: uniform bottom with mounds or depressions 0-3 m; outcrops)","M - deposition (dusting <1 cm, thin 1-5 cm, thick > 5 cm)","M - texture (sorting, packing, density: occasional, scattered, contiguous, pavement)",...,CPC?,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,LTIME,LDATE,LAT,LON,CDATE
dte,,,,,,,,,,,,,,,,,,,,,
2019-02-13 20:10:29,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:10:29,Seamount - EI (SPG7),Flat sandy bottom,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:10:42,LAT -26.1875 LON -102.996 Name: 2019-02-13...,-26.1875,-102.996,TIME 20:10:29 CTD_TEMP 4.4153 CT...
2019-02-13 20:11:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:11:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:11:18,LAT -26.1875 LON -102.996 Name: 2019-02-13...,-26.1875,-102.996,TIME 20:11:17 CTD_TEMP 4.4285 CT...
2019-02-13 20:12:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:12:17,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 20:12:18,LAT -26.1874 LON -102.995 Name: 2019-02-13...,-26.1874,-102.995,TIME 20:12:17 CTD_TEMP 4.5002 CT...
2019-02-13 20:13:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:13:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - dusting,NaN,...,Y,NaN,NaN,NaN,NaN,2019-02-13 20:13:18,LAT -26.1873 LON -102.995 Name: 2019-02-13...,-26.1873,-102.995,TIME 20:13:17 CTD_TEMP 4.5622 CT...
2019-02-13 20:14:17,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,20:14:17,Seamount - EI (SPG7),NaN,rock,NaN,irregular with exposed volcanic bedrock,sediment - thin,NaN,...,Y,NaN,NaN,NaN,NaN,2019-02-13 20:14:18,LAT -26.1872 LON -102.995 Name: 2019-02-13...,-26.1872,-102.995,TIME 20:14:17 CTD_TEMP 4.6181 CT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-13 21:39:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:39:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:39:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,-26.1813,-102.976,TIME 21:39:18 CTD_TEMP 14.3061 CT...
2019-02-13 21:40:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:40:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:40:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,-26.1813,-102.976,TIME 21:40:18 CTD_TEMP 14.3223 CT...
2019-02-13 21:41:18,EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...,2/13/2019,21:41:18,Seamount - EI (SPG7),NaN,coarse sand,NaN,regular,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019-02-13 21:41:18,LAT -26.1813 LON -102.976 Name: 2019-02-13...,-26.1813,-102.976,TIME 21:41:18 CTD_TEMP 14.3457 CT...


In [125]:
#Now getting the series with the latitude and longitude data, but they'll need some cleaning up.

Hab_37 = []
num = 0
while num < 89:

    Hab37_2 = CTD37.iloc[CTD37.index.get_loc(Hab37.index[num], method='nearest')][[1, 4, 5, 6]]
    Hab_37.append(Hab37_2)
    num = num +1
print(Hab_37)

[TIME         20:10:29
CTD_TEMP       4.4153
CTD_DEPTH      880.27
CTD_SAL       34.3888
Name: 2019-02-13 20:10:29, dtype: object, TIME         20:11:17
CTD_TEMP       4.4285
CTD_DEPTH      879.74
CTD_SAL       34.3865
Name: 2019-02-13 20:11:17, dtype: object, TIME         20:12:17
CTD_TEMP       4.5002
CTD_DEPTH       873.5
CTD_SAL       34.3756
Name: 2019-02-13 20:12:17, dtype: object, TIME         20:13:17
CTD_TEMP       4.5622
CTD_DEPTH      857.95
CTD_SAL       34.3667
Name: 2019-02-13 20:13:17, dtype: object, TIME         20:14:17
CTD_TEMP       4.6181
CTD_DEPTH      838.91
CTD_SAL       34.3583
Name: 2019-02-13 20:14:17, dtype: object, TIME         20:15:17
CTD_TEMP       4.6675
CTD_DEPTH      825.69
CTD_SAL       34.3514
Name: 2019-02-13 20:15:17, dtype: object, TIME         20:16:17
CTD_TEMP       4.6824
CTD_DEPTH      825.09
CTD_SAL       34.3502
Name: 2019-02-13 20:16:17, dtype: object, TIME         20:17:17
CTD_TEMP       4.6906
CTD_DEPTH      822.65
CTD_SAL       34.3492
N

In [149]:
#Only required a small modification, but now it's good. These loops are definitely getting easier, that's for sure.

num = 0
Hab37['CTIME'] = ''
Hab37['TEMP'] = ''
Hab37['DEPTH'] = ''
Hab37['SAL'] = ''

    
while num <89:


    st = str(Hab37['CDATE'][num])
    spl = st.split('\n')
    del(spl[-1])
    spl = pd.DataFrame(spl)
    spl = spl[0].str.strip(' , T, I, M, E, C, D, P, H, S, A, L, _')
    Hab37['CTIME'][num] = spl[0]
    Hab37['TEMP'][num] = spl[1]
    Hab37['DEPTH'][num] = spl[2]
    Hab37['SAL'][num] = spl[3]
   
    num = num + 1
print(Hab37)

<ipython-input-149-5d6c47864584>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['CTIME'][num] = spl[0]
<ipython-input-149-5d6c47864584>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['TEMP'][num] = spl[1]
<ipython-input-149-5d6c47864584>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab37['DEPTH'][num] = spl[2]
<ipython-input-149-5d6c47864584>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

                                                                 Image  \
dte                                                                      
2019-02-13 20:10:29  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:11:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:12:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:13:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 20:14:17  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
...                                                                ...   
2019-02-13 21:39:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:40:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:41:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:42:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   
2019-02-13 21:43:18  EPIC_Cruise_MR180603_6KCDT0037-20190213-194333...   

                          DATE      T

In [164]:
CTD39 = CTD39.loc[~CTD39.index.duplicated(keep = 'first')]

In [165]:
#Now to start on the CTD data

Hab_39 = []
num = 0
while num < 150:

    Hab39_2 = CTD39.index[CTD39.index.get_loc(Hab39.index[num], method='nearest')]
    Hab_39.append(Hab39_2)
    num = num +1
print(Hab_39)

[Timestamp('2019-02-16 20:03:58'), Timestamp('2019-02-16 20:04:58'), Timestamp('2019-02-16 20:05:58'), Timestamp('2019-02-16 20:06:58'), Timestamp('2019-02-16 20:07:58'), Timestamp('2019-02-16 20:08:58'), Timestamp('2019-02-16 20:09:58'), Timestamp('2019-02-16 20:10:58'), Timestamp('2019-02-16 20:11:58'), Timestamp('2019-02-16 20:12:58'), Timestamp('2019-02-16 20:13:58'), Timestamp('2019-02-16 20:14:58'), Timestamp('2019-02-16 20:15:58'), Timestamp('2019-02-16 20:16:59'), Timestamp('2019-02-16 20:17:59'), Timestamp('2019-02-16 20:18:59'), Timestamp('2019-02-16 20:19:59'), Timestamp('2019-02-16 20:20:59'), Timestamp('2019-02-16 20:21:59'), Timestamp('2019-02-16 20:22:59'), Timestamp('2019-02-16 20:23:59'), Timestamp('2019-02-16 20:24:59'), Timestamp('2019-02-16 20:25:59'), Timestamp('2019-02-16 20:26:59'), Timestamp('2019-02-16 20:27:59'), Timestamp('2019-02-16 20:28:59'), Timestamp('2019-02-16 20:29:59'), Timestamp('2019-02-16 20:30:59'), Timestamp('2019-02-16 20:31:59'), Timestamp('20

In [166]:
Hab39['CTIME'] = Hab_39

In [167]:
Hab_39 = []
num = 0
while num < 150:

    Hab39_2 = CTD39.iloc[CTD39.index.get_loc(CTD39.index[num], method='nearest')][[1, 4, 5, 6]]
    Hab_39.append(Hab39_2)
    num = num +1
print(Hab_39)

[TIME         19:39:21
CTD_TEMP      25.5462
CTD_DEPTH       12.68
CTD_SAL       31.7697
Name: 2019-02-16 19:39:21, dtype: object, TIME         19:39:22
CTD_TEMP      25.5477
CTD_DEPTH       13.02
CTD_SAL       31.9262
Name: 2019-02-16 19:39:22, dtype: object, TIME         19:39:23
CTD_TEMP       25.549
CTD_DEPTH       13.51
CTD_SAL       32.2106
Name: 2019-02-16 19:39:23, dtype: object, TIME         19:39:24
CTD_TEMP      25.5471
CTD_DEPTH       14.15
CTD_SAL       32.0321
Name: 2019-02-16 19:39:24, dtype: object, TIME         19:39:25
CTD_TEMP      25.5451
CTD_DEPTH       14.46
CTD_SAL       31.9642
Name: 2019-02-16 19:39:25, dtype: object, TIME         19:39:26
CTD_TEMP      25.5462
CTD_DEPTH       14.46
CTD_SAL       31.9435
Name: 2019-02-16 19:39:26, dtype: object, TIME         19:39:27
CTD_TEMP      25.5452
CTD_DEPTH       14.02
CTD_SAL       31.8888
Name: 2019-02-16 19:39:27, dtype: object, TIME         19:39:28
CTD_TEMP      25.5455
CTD_DEPTH       13.34
CTD_SAL       31.6655
N

In [187]:
#Only required a small modification, but now it's good. These loops are definitely getting easier, that's for sure.

num = 0
Hab39['CTIME'] = ''
Hab39['TEMP'] = ''
Hab39['DEPTH'] = ''
Hab39['SAL'] = ''

    
while num <150:

    Hab39['CDATE'] = Hab_39
    st = str(Hab39['CDATE'][num])
    spl = st.split('\n')
    del(spl[-1])
    spl = pd.DataFrame(spl)
    spl = spl[0].str.strip(' , T, I, M, E, C, D, P, H, S, A, L, _')
    Hab39['CTIME'][num] = spl[0]
    Hab39['TEMP'][num] = spl[1]
    Hab39['DEPTH'][num] = spl[2]
    Hab39['SAL'][num] = spl[3]
   
    num = num + 1
print(Hab39)

<ipython-input-187-4c31c093ce19>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab39['CTIME'][num] = spl[0]
<ipython-input-187-4c31c093ce19>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab39['TEMP'][num] = spl[1]
<ipython-input-187-4c31c093ce19>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Hab39['DEPTH'][num] = spl[2]
<ipython-input-187-4c31c093ce19>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

                                                                 Image  \
dte                                                                      
2019-02-16 20:03:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:04:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:05:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:06:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 20:07:58  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
...                                                                ...   
2019-02-16 22:31:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:32:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:33:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:34:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   
2019-02-16 22:35:02  EPIC_Cruise_MR180603_6KCDT0039_6KCDT0039-20190...   

                          DATE      T

In [188]:
Hab37.to_csv('HAB37.csv')

In [189]:
Hab39.to_csv('HAB39.csv')

In [ ]:
#Well that was painful and complicated but it's done and they're saved as .csv's so I can access them easily whenever.